# ABC Pharmaceuticals

ABC Pharmaceuticals is embarking on a process to model the inputs and potential revenue generated from our three products given different views on the demand and costs of producing these. This code is for internal analysis of the data coming from our production facility to determine if we can build a model that will allow us to have a clear view of the feed requirements for different forward looking scenarios.

### Required Imports

In [ ]:
import pandas as pd
import numpy as np
import joblib
import nbformat
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import differential_evolution
from sklearn.model_selection import train_test_split

# Data Exploration

ABC Chemicals produces three primary products. To do this, they use 4 raw feed inputs into three operational units. The feed inputs are liquids and have units of measure of mega-litres (Ml), while the three primary products are measured in kilograms (kg). Each feed input is processed through a unit. Due to the type of process occurring, it is possible to also produce by-products. The past 4 years worth of data is in the excel document (Data Inputs.xlsx)

In [ ]:
input_data = pd.read_excel("Data_Inputs.xlsx")
input_data.head()

As a first analysis, let us explore the data to ensure that it is complete and usable

In [ ]:
#Count same number of entries for all
print(f"Counting rows per feature:", input_data.count())

#Check for nulls 
print(f"Counting number of nulls per feature:", input_data.where(input_data.isna()).count())

# #Check for correct datatypes 
print(f"DataType check:",input_data.dtypes)

Let us continue and plot out the inputs and outputs from the data frame as time series plots

In [ ]:
input_data["Timestamp"] = pd.to_datetime(input_data["Timestamp"])

fig = go.Figure()
for title in ["Product1", "Product2", "Product3", "ByProduct"]:
    fig.add_trace(
        go.Scatter(
            mode='lines',
            x=input_data["Timestamp"],
            y=input_data[title],
            hoverinfo='skip',
            name=title,
            # line=dict(color='Black',
            #         width=2)
        )
    )
fig.show()


fig = go.Figure()
for title in ["Raw1", "ByProduct_From_1_to_2", "Raw2", "ByProduct_From_1_to_3", "Raw3_1", "Raw3_2"]:
    fig.add_trace(
        go.Scatter(
            mode='lines',
            x=input_data["Timestamp"],
            y=input_data[title],
            hoverinfo='skip',
            name=title,
            # line=dict(color='Black',
            #         width=2)
        )
    )
fig.show()

Next, let's go through each of the unit operations and see how the input data corresponds to the output from the unit

In [ ]:
# Unit 1

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=input_data["Raw1"],
        y=input_data["Product1"],
        hoverinfo='skip',
        name="Product1",
        # line=dict(color='Black',
        #         width=2)
    )
)
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=input_data["Raw1"],
        y=input_data["ByProduct"],
        hoverinfo='skip',
        name="ByProduct",
        # line=dict(color='Black',
        #         width=2)
    )
)

fig.update_layout({'title': "Byprodut and Product1 from Unit 1 as a function of Feed Raw1",
                   "xaxis": {'title': "Raw1 (Ml)"},
                   "yaxis": {'title': "Product1 (kg), Byproduct (Ml)"}})

fig.show()

Next, unit 2. Unit 2 receives 2 inputs and therefore we need to consider each of them to see what the relationship might be

In [ ]:
# TO DO: Replicate analysis above for unit 2


And finally for unit3

In [ ]:
# TO DO: Replicate analysis above for unit 3

Just for completeness, let us do a correlation matrix as well

In [ ]:
# TO DO: Plot a correlation heatmap

# Data Cleaning

Before we start cleaning the data, we will split the data into the inputs and outputs for each unit operation. We will also add the timestamp as the index.

In [ ]:
unit_1 = input_data[["Raw1", "ByProduct", "Product1"]]
unit_1.index = input_data["Timestamp"]
unit_2 = input_data[["ByProduct_From_1_to_2", "Raw2", "Product2"]]
unit_2.index = input_data["Timestamp"]
unit_3 = input_data[["ByProduct_From_1_to_3", "Raw3_1", "Raw3_2", "Product3"]]
unit_3.index = input_data["Timestamp"]

#Checking output
unit_1.describe()

We need to clean the data before testing any models. An external source has provided code that can be used to do this. The function is is the helper_code file, and is called mahalanobis_outlier_removal. We have not yet had time to implement this approach.

In [ ]:
# TO DO: Implement the mahalanobis_outlier_removal function
#clean_unit1 = mahalanobis_outlier_removal()

# Unit Model Building

We need to create training and testing sets.

In [ ]:
# TO DO: split the data into train and test datasets. For now we don't need to consider an out of time or validation dataset
# unit_1_train, unit_1_test = 

Now we can train a model for each of the four units

In [ ]:
# TO DO: train the four models

Let's evaluate the models that we have built

In [ ]:
# TO DO: Evaluate MSE, MAE, R2, MAPE and RMSE for each of the models, also get parity plots and residuals

Now we can build a final version of the model that is built with all the data

In [ ]:
# TO DO: Build and save final models

# Plant Model Build

The next step in the process is to build a model for the whole of the plant. This will take as inputs, the feed rates of the 4 raw feeds, as well as the fraction of byproduct that is directed to each of unit2 and unit3. 

In [ ]:
def plant(plant_input: pd.DataFrame):
    # TO DO:
    # Update code to allow the byproduct to have an excess stream as well.

    plant_input["byproduct_to_unit2"] = plant_input["byproduct_to_unit2"]/(plant_input["byproduct_to_unit2"] + plant_input["byproduct_to_unit3"])
    plant_input["byproduct_to_unit3"] = plant_input["byproduct_to_unit3"]/(plant_input["byproduct_to_unit2"] + plant_input["byproduct_to_unit3"])

    # plant_input : 6 input feeds, and 2 values for the percentages of each feed that goes to unit 2 and 3
    reg_product1 = joblib.load("unit_models/{}.joblib".format("product_1"))
    reg_byproduct = joblib.load("unit_models/{}.joblib".format("byproduct"))
    reg_product2 = joblib.load("unit_models/{}.joblib".format("product_2"))
    reg_product3 = joblib.load("unit_models/{}.joblib".format("product_3"))

    product1 = reg_product1.predict(plant_input[["Raw1"]]).reshape([len(plant_input[["Raw1"]]),])
    byproduct = reg_byproduct.predict(plant_input[["Raw1"]])

    plant_input["byproduct"] = byproduct
    plant_input["ByProduct_From_1_to_2"] = plant_input["byproduct"]*plant_input["byproduct_to_unit2"]
    plant_input["ByProduct_From_1_to_3"] = plant_input["byproduct"]*plant_input["byproduct_to_unit3"]


    product2 = reg_product2.predict(plant_input[["ByProduct_From_1_to_2", "Raw2"]])
    product3 = reg_product3.predict(plant_input[["ByProduct_From_1_to_3", "Raw3_1", "Raw3_2"]])

    return product1, product2, product3, plant_input["byproduct"] - plant_input["ByProduct_From_1_to_2"] - plant_input["ByProduct_From_1_to_3"]

# Plant Optimization

Now we need to define the objective function to use for the optimization

In [ ]:
def objective_function(product1, product2, product3, byproduct,
                       product_sell_prices, product_demand, unmet_demand_penalty,
                       storage_costs, discard_costs, detailed=False):
    
    # TO DO: Build out the objective function required for the optimization
    
    pass

Now we need to build the optimizer. Based on prior experience, the differential evolution algorithm has been selected for the optimizer, however, this is not required

In [ ]:
def optimizer(
        bounds, 
        opt_params, 
        PRODUCT_SELL_PRICE, 
        PRODUCT_DEMAND, 
        PRODUCT_UNMET_DEMAND_PENALTY,
        PRODUCT_STORAGE_COST, 
        PRODUCT_DISCARD_COST
        ):
    
    def model(X_in):
        if len(X_in.shape) == 1:
            X_in = X_in.reshape([1,6])
        X_in = pd.DataFrame(X_in, columns=["Raw1","Raw2","Raw3_1","Raw3_2","byproduct_to_unit2","byproduct_to_unit3"])
        if np.isnan(X_in.iloc[0,0]):
            return np.nan
        product1, product2, product3, byproduct = plant(X_in)

        objective = objective_function(product1, product2, product3, byproduct,
                       PRODUCT_SELL_PRICE, PRODUCT_DEMAND, PRODUCT_UNMET_DEMAND_PENALTY,
                       PRODUCT_STORAGE_COST, PRODUCT_DISCARD_COST)
        return objective.iloc[0]

    result = differential_evolution(model, bounds=bounds, **opt_params)

    return result

# Final Output

We need to capture the outputs from all the scenarios

# Visualisaction and Analysis

Let us get some visualisations of the outputs, or else get pretty printed versions that compare the outputs of the different scenarios